In [ ]:
!pip install git+https://github.com/andrianllmm/tagalog-stemmer.git@main
!pip install pyspellchecker

  Cloning https://github.com/andrianllmm/tagalog-stemmer.git (to revision main) to /tmp/pip-req-build-fd1pitox
  Running command git clone --filter=blob:none --quiet https://github.com/andrianllmm/tagalog-stemmer.git /tmp/pip-req-build-fd1pitox
  Resolved https://github.com/andrianllmm/tagalog-stemmer.git to commit b5babfd4caebf8a8f480f8adab9f1c97f42a3baa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tglstemmer: filename=tglstemmer-0.0.1-py3-none-any.whl size=146667 sha256=81c863dd4f2a7ae79608f5c501bb71263724d08b3bfbb6d20f75965864bda5e5
  Stored in directory: /tmp/pip-ephem-wheel-cache-efmggn5f/wheels/8c/bb/43/2bad866e363692fb17ed779ce8517901b301c705dc7a438b33
Successfully built tglstemmer
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 64.7 MB/s eta 0:00:00


In [ ]:
import csv
import re
import operator
import string
from tglstemmer import stemmer
from spellchecker import SpellChecker
spell = SpellChecker()

mispellings = dict()
with open("mispelling.csv") as h:
  reader = csv.reader(h)
  for row in reader:
    mispellings[row[0]] = row[1]

tweets = []

#clean up
def remove_non_ascii(a_str):
  ascii_chars = set(string.printable)

  return "".join(
      filter(lambda x: x in ascii_chars, a_str)
  )

def replace_mispellings(sentence):
  wordlist = sentence.split()
  for word in wordlist:
    index = wordlist.index(word)
    if word in mispellings:
      word = mispellings.get(word)
    wordlist[index] = word
  sentence = ' '.join(wordlist)
  return sentence

def remove_punctuation(sentence):
  punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  no_punct = ""
  for char in sentence:
    if char not in punctuations:
        no_punct = no_punct + char
  return no_punct

def correct_mispellings(sentence):
    wordlist = sentence.split()
    new_sentence = []
    for word in wordlist:
        corrected_word = spell.correction(word)
        if corrected_word is None:
            corrected_word = word
        new_sentence.append(corrected_word)
    return " ".join(new_sentence)

#importing list of tweets
with open("newtwitter.csv") as f:
  reader = csv.reader(f)
  next(reader)

  for row in reader:
    tweet = dict()
    tweet['orig'] = row[0]

    if re.match(r'^RT.*', tweet['orig']):
      continue
    tweet['clean'] = remove_punctuation(tweet['orig'])
    tweet['clean'] = correct_mispellings(tweet['clean'])
    tweet['clean'] = replace_mispellings(tweet['clean'])
    tweet['clean'] = remove_non_ascii(tweet['clean'])
    tweet['clean'] = tweet['clean'].lower()
    tweets.append(tweet)

lexicon_eng = dict()
lexicon_fil = dict()

#importing english lexicon
with open("lexicon_eng.csv") as f:
  reader = csv.reader(f)
  for row in reader:
    lexicon_eng[row[0]] = int(row[1])

#importing filipino lexicon
with open("lexicon_fil.csv") as g:
  reader = csv.reader(g)
  for row in reader:
    lexicon_fil[row[0]] = float(row[1])

#scoring tweets based on lexicon
for tweet in tweets:
  score = 0
  for word in tweet["clean"].split():
    if word in lexicon_eng:
      score = score + lexicon_eng[word]
    else:
      #if not an english word, then assume it is a filipino and stem to be able to score
      stemword = stemmer.get_stem(word)
      if word in lexicon_fil:
        score = score + lexicon_fil[word]
      else:
        score = score

  #categorizing if tweet is positive or negative
  tweet['score'] = score
  if (score > 0):
    tweet['sentiment'] = 'positive'
  elif (score < 0):
      tweet['sentiment'] = 'negative'
  else:
      tweet['sentiment'] = 'neutral'

#printing results
total = float(len(tweets))
num_pos = sum([1 for t in tweets if t['sentiment'] == 'positive'])
num_neg = sum([1 for t in tweets if t['sentiment'] == 'negative'])
num_neu = sum([1 for t in tweets if t['sentiment'] == 'neutral'])
print ("Positive: %5d (%.1f%%)" % (num_pos, 100.0 * (num_pos/total)))
print ("Negative: %5d (%.1f%%)" % (num_neg, 100.0 * (num_neg/total)))
print ("Neutral:  %5d (%.1f%%)" % (num_neu, 100.0 * (num_neu/total)))

Positive:    18 (50.0%)
Negative:     9 (25.0%)
Neutral:      9 (25.0%)


In [ ]:
from spellchecker import SpellChecker
spell = SpellChecker()

# find those words that may be misspelled
sentence = "cmputr watr study wrte"

def correct_mispellings(sentence):
    wordlist = sentence.split()
    new_sentence = []
    for word in wordlist:
        corrected_word = spell.correction(word)
        if corrected_word is None:
            corrected_word = word  # fallback to original word if correction is None
        new_sentence.append(corrected_word)
    return " ".join(new_sentence)




print(correct_mispellings(sentence))

computer water study write
